In [1]:
using QuantumCumulants
using OrdinaryDiffEq, ModelingToolkit
# using Optim
# using SparseArrays
# using DifferentialEquations
using PyPlot
# using NPZ
# using ArgParse
# using BenchmarkTools
using DelimitedFiles
# using BeepBeep
import LinearAlgebra as la
using Statistics

In [2]:
multiplier = 1
M_1 = 10900
M_2 = 1750
# M_3 = 0.175
y1 = 2.94e-3*multiplier
y2 = 1.76e-2*multiplier/2/pi

@cnumbers ω1 ω2 ωc g1 g2 g3 λ1 λ2 λc κ1 κ2 κc α1 α2 β  # 2-magnon, 2-photon
@cnumbers Ω1 Ω2 Ωc 
h1 = FockSpace(:cavity);h2 = FockSpace(:cavity);hc = FockSpace(:cavity)
h=h1⊗h2⊗hc
# Define the fundamental operators
@qnumbers b1::Destroy(h,1) b2::Destroy(h,2) bc::Destroy(h,3)
#            magnon PY       magnon YIG        resonator

Ham = ω1*(1-im*α1)*(b1'*b1) + ω2*(1-im*α2)*(b2'*b2) + ωc*(1-im*β)*(bc'*bc) + g1*((b1'*bc)+(bc'*b1)) + g2*((bc'*b2)+(b2'*bc)) + Ω1*( b1'+b1) + Ω2*(b2'+b2) + Ωc*(bc'+bc)
# Collapse operators
J = [b1,b2,bc]; rates = [λ1,λ2,λc]
# Derive a set of equations
ops = [b1,b2,bc]; eqs = meanfield(ops,Ham,J;rates=rates,order=1)

# Complete equations
eqs_completed = complete(eqs);
@named sys = ODESystem(eqs_completed);
display(sys)
A = calculate_jacobian(sys); B=[eqs_completed[1].rhs.dict[Ω1] * Ω1; eqs_completed[2].rhs.dict[Ω2] * Ω2; eqs_completed[3].rhs.dict[Ωc] * Ωc];
Ainv=inv(A); X=Ainv*B; b1=X[1]; b2=X[2]; bc=X[3];

Model sys:
Equations (3):
  3 standard: see equations(sys)
Unknowns (3): see unknowns(sys)
  var"⟨b1⟩"(t)
  var"⟨b2⟩"(t)
  var"⟨bc⟩"(t)
Parameters (14): see parameters(sys)
  Ωc
  ω1
  ωc
  α2
  ⋮

In [122]:
ω2n = H -> y2 * (H*(H+M_2))^.5
ω1n = H -> y1 * (H*(H+M_1))^.5

function main_calc_real_part(Hlist,ωcn,g1n,g2n,λ1n,λ2n,λcn,α1n,α2n,βn)
    # println("Running main_calc_real_part")
    occupationList1 = Float64[]; occupationList2 = Float64[]; occupationList3 = Float64[];
    for H in Hlist
        # if H
        An=substitute( A, Dict(ω1=>ω1n(H),ωc=>ωcn,ω2=>ω2n(H),g1=>g1n,g2=>g2n,λ1=>λ1n,λ2=>λ2n,λc=>λcn,α1=>α1n,α2=>α2n,β=>βn))
        data0 = 1im .* An
        data1 = la.eigen(data0)
        datar=la.real(data1.values)
        sort!(datar,rev=true)
        # datar = filter(x -> x >= minimum(locs) && x <= maximum(locs), datar)
        if length(datar) == 1
            datar = [datar[1], datar[1], datar[1]]
        elseif length(datar) == 0
            # println(H)# = [datar[1], datar[1]]
            datar = [ω2n, ω2n, ω2n]
        elseif length(datar) == 2
            datar = [datar[1], datar[2], datar[2]]
        end
        # println(size(datar))
        # r1n=datar[1]; r2n=datar[2];# r3n=datar[3]; 
        #print(An)
        push!(occupationList1, datar[1]); push!(occupationList2, datar[2]); push!(occupationList3, datar[3]);
    end
    occupationList = [occupationList1 occupationList2 occupationList3]

    return occupationList

end

function main(type, optimized_params)

    println("Running main for $type")

    # type = "strong"
    root = joinpath(pwd(),"data","yig_t_sweep_new")
    # Read the CSV file into a DataFrame
    # file_path = joinpath(root,"strong_peaks_widths.csv")
    # file_path = joinpath(root, "peaks_widths", "$type"*"_peaks_widths.csv")
    file_path_full = joinpath(root,"$type.csv")
    # df = readdlm(file_path, ',', Float64, '\n',skipstart=1)
    full_data = readdlm(file_path_full,',',Float64,'\n')

    # Display the first few rows of the DataFrame
    frequencies = full_data[2:end,1];
    s21 = full_data[2:end,2:end];
    # locs = df[:,1:2] * 2e9 * pi;
    # locs = sort(locs, dims=2)
    Hlist = full_data[1,2:end];

    # for i in 1:length(Hlist)
    s21_H = s21
    # if i == 150
    #     println("s21_H: ", s21_H)
    # end
    s21_H[ (s21_H .- mean(s21_H)) ./ std(s21_H) .> 3 ] .= 0
    s21 = s21_H
    # end

    function inter(Hlist, ωcn, params)
        n_field = length(Hlist)
        n_freq = length(frequencies)
        array = zeros(n_field, n_freq)
        data_points = main_calc_real_part(Hlist,ωcn, params...)
        for i in 1:3
            for (idx,point) in enumerate(data_points[:,i])
                index = findmin(abs.(frequencies .- point))[2]
                array[idx,index] += 1
            end
        end
        array = array'  # Transpose the array to match the dimensions of the s21 array
        
        sq_error = (s21.+array).^2
        
        return sum(sq_error)
    end

    root = joinpath(pwd(),"results")

   
    ωcn = optimized_params[1]
    optimized_params = optimized_params[2:end]

    objective(params) = inter(Hlist, ωcn, params)

    # # Perform the optimization
    # lower = [5.06, 0, 0]
    # upper = [3.3, 1, 1]
    # inner_optimizer = LBFGS()
    # result = optimize(objective,lower,upper,initial_params,Fminbox(inner_optimizer))
    # result = optimize(objective,optimized_params,inner_optimizer)
    # Extract optimized parameters
    # optimized_params = Optim.minimizer(result)
    # optimized_params = [0.05, 0.19]

    println("Optimized parameters: ", ωcn, optimized_params)
    #Numerical calculations of dispersion spectra for case-1 (J > Γ)

    occupationList = main_calc_real_part(Hlist,ωcn,optimized_params...)

    return (Hlist, frequencies, s21, occupationList, optimized_params)
end


function parallel_main(files)
    Threads.@threads for file in files
        main(file)
    end
    return nothing
end

function serial_main(files, optimized_params)
    for i in 1:length(files)
        main(files[i], optimized_params[i])
    end
    return nothing
end

function plot_multiple_calculations(params)
    # files = keys(params)
    files = sort(collect(keys(params)))
    num_plots = length(files)
    nrows = 2
    ncols = max(ceil(Int, num_plots / nrows),2)
    println("ncols:",ncols)

    size = 3

    # Create a figure and a grid of subplots
    # fig, axes = subplots()

    # fig = figure(figsize=(2ncols,2nrows), sharey=true, sharex=true)
    fig, axes = subplots(nrows, ncols, figsize=(size*ncols,size*nrows), sharey=true, sharex=true)
    idx = 6

    

    
    for (i, file) in enumerate(files)
    # for file in files
        param = params[file]
        # println("Plotting $file with params $param")
        # return main(file, param)
        Hlist, frequencies, s21, occupationList, optimized_params = main(file, param)

        s21[ abs.(s21 .- mean(s21)) ./ std(s21) .> 7 ] .= 0
        # if idx == 5
        #     ax = axes[7]
        # else
        #     ax = axes[(idx+2)%7]
        #     # ax = axes
        # end
        # idx = (idx+2)%7
        # ax = fig.add_subplot(nrows, ncols, i+1, frameon=true)  # Create a subplot for each file

        ax = axes[i]

        t = round(parse(Float64, split(file, "_")[3]), digits=3)

        im = ax.pcolormesh(Hlist, frequencies, s21, cmap=:inferno_r)
        ax.plot(Hlist, occupationList[:,1], "w",alpha=.5)
        ax.plot(Hlist, occupationList[:,2], "w",alpha=.5)
        ax.plot(Hlist, occupationList[:,3], "w",alpha=.5)
        tt = Int64(t*1e3)
        ax.text(1250, 5.5, "t = $tt μm", color="white", fontsize=10, ha="right")
        optimized_params = round.(optimized_params, digits=2)
        # ax.set_title("t = $t"*raw"$\mu$m"*";\nParams = $optimized_params")
        # if i==4
        #     ax.set_xlabel("Magnetic Field (Oe)")
        # end
        # if i==1 || i==2
        #     ax.set_ylabel("Frequency (GHz)")
        # end
        # ax.set_ylabel("Frequency (GHz)")
        # ax.set_ylim(2.75, 3.95)
        ax.set_ylim(4.3,6)
        # if i % ncols == 0
            # fig.colorbar(im, ax=ax)
        # end
    end
    # fig.colorbar(im, ax=axes[end-1], orientation="horizontal")
    ax_big = fig.add_subplot(111,frameon=false)  # Create a big subplot

    # Turn off ax_bigis lines and ticks of the big subplot
    ax_big.spines["top"].set_color("none")
    ax_big.spines["bottom"].set_color("none")
    ax_big.spines["left"].set_color("none")
    ax_big.spines["right"].set_color("none")
    ax_big.tick_params(labelcolor="none",top=false, bottom=false, left=false, right=false)

    # Set common labels
    ax_big.set_xlabel("H (Oe)",fontsize=12)
    ax_big.set_ylabel("Frequency (GHz)\n",fontsize=12)
    # ax_big.plot([Hlist[1],Hlist[end]], [frequencies[1],frequencies[end]], color="none")
    # If there are unused subplots, hide them
    for j in (num_plots+1):length(axes)
        axes[j].axis("off")
    end

    tight_layout()
    savefig("images\\combined_plots.png",dpi=300,bbox_inches="tight")  # Save the figure with a tight layout
    # show()
    println("Saved figure to combined_plots.png")
    close(fig)  # Close the figure if you don't want to display it
end

plot_multiple_calculations (generic function with 1 method)

In [123]:
params = Dict( # ωcn  g1n  g2n λ1n  λ2n  λcn  α1  α2  β
             "yig_t_0.000" => [5.09, .13, 0.0,.01,.01,.01,2e-2,1e-5,1e-5],  
             "yig_t_0.005" => [5.06, .13, 0.04, .01,.01,.01,2e-2,1e-5,1e-5],  
             "yig_t_0.013" => [5.01, .13, 0.075,.01,.01,.01,2e-2,1e-5,1e-5], 
             "yig_t_0.027" => [5.01, .14, 0.12,.01,.01,.01,2e-2,1e-5,1e-5],  
             "yig_t_0.040" => [5.04, .16, 0.13,.01,.01,.01,2e-2,1e-5,1e-5], 
             "yig_t_0.053" => [5.01, .16, 0.15,.01,.01,.01,2e-2,1e-5,1e-4], 
             "yig_t_0.067" => [5.02, .16, 0.18,.01,.01,.01,2e-2,3e-5,1e-4],  
             "yig_t_0.100" => [5.01, .2, 0.25,.01,.01,.01,2e-2,1e-5,1e-4], 
             )

println("Threads allocated: ", Threads.nthreads())

plot_multiple_calculations(params)
# beep(4)

Threads allocated: 1
ncols:4
Running main for yig_t_0.000
Optimized parameters: 5.09[0.13, 0.0, 0.01, 0.01, 0.01, 0.02, 1.0e-5, 1.0e-5]
Running main for yig_t_0.005
Optimized parameters: 5.06[0.13, 0.04, 0.01, 0.01, 0.01, 0.02, 1.0e-5, 1.0e-5]
Running main for yig_t_0.013
Optimized parameters: 5.01[0.13, 0.075, 0.01, 0.01, 0.01, 0.02, 1.0e-5, 1.0e-5]
Running main for yig_t_0.027
Optimized parameters: 5.01[0.14, 0.12, 0.01, 0.01, 0.01, 0.02, 1.0e-5, 1.0e-5]
Running main for yig_t_0.040
Optimized parameters: 5.04[0.16, 0.13, 0.01, 0.01, 0.01, 0.02, 1.0e-5, 1.0e-5]
Running main for yig_t_0.053
Optimized parameters: 5.01[0.16, 0.15, 0.01, 0.01, 0.01, 0.02, 1.0e-5, 0.0001]
Running main for yig_t_0.067
Optimized parameters: 5.02[0.16, 0.18, 0.01, 0.01, 0.01, 0.02, 3.0e-5, 0.0001]
Running main for yig_t_0.100
Optimized parameters: 5.01[0.2, 0.25, 0.01, 0.01, 0.01, 0.02, 1.0e-5, 0.0001]
Saved figure to combined_plots.png


In [5]:
type = "yig_t_0.02"

root = joinpath(pwd(),"data","yig_t_sweep_outputs")
# file_path = joinpath(root, "peaks_widths", "$type"*"_peaks_widths.csv")
file_path_full = joinpath(root,"$type.csv")
# df = readdlm(file_path, ',', Float64, '\n',skipstart=1)
full_data = readdlm(file_path_full,',',Float64,'\n')

# Display the first few rows of the DataFrame
frequencies = full_data[2:end,1] * 2e9 * pi;
s21 = full_data[2:end,2:end];
# locs = df[:,1:2] * 2e9 * pi;
# locs = sort(locs, dims=2)
Hlist = full_data[1,2:end];

In [6]:

function calculate_coupling_strengths(Hlist, params)
    # files = keys(params)
    data_points = main_calc_real_part(Hlist,params...)
    noccupationList1 = data_points[:,1]; noccupationList2 = data_points[:,2]; noccupationList3 = data_points[:,3];
    d_noccupationList1 = diff(noccupationList1)
    d_noccupationList2 = diff(noccupationList2)
    d_noccupationList3 = diff(noccupationList3)
    diff_of_deriv1 = abs.(d_noccupationList1 .- d_noccupationList2)
    minimum_deriv1 = findmin(diff_of_deriv1)[2]
    diff_of_deriv2 = abs.(d_noccupationList2 .- d_noccupationList3)
    minimum_deriv2 = findmin(diff_of_deriv2)[2]

    coupling_strength1 = abs(noccupationList1[minimum_deriv1] - noccupationList2[minimum_deriv1])
    coupling_strength2 = abs(noccupationList2[minimum_deriv2] - noccupationList3[minimum_deriv2])

    return coupling_strength1, coupling_strength2
end

println(calculate_coupling_strengths(Hlist, params[type]))

KeyError: KeyError: key "yig_t_0.02" not found

In [7]:
function coupling_strengths(index)
#          1    2    3    4    5    6   7   8   9
    # symbols = [α1  α2]
    symbols = [ωc  g1  g2  λ1  λ2  λc  α1  α2  β]

    param_array = [3.175, 1.6, 0.2135,.1,.1,.01,.1,.1,.1]
    alpha_array = range(0, 1000, length=500*2)
    g1_array = zeros(size(alpha_array))
    g2_array = zeros(size(alpha_array))

    for (idx,alpha) in enumerate(alpha_array)
        param_array2 = param_array[1:end]
        param_array2[index] = alpha
        gs = calculate_coupling_strengths(Hlist, param_array2)
        g1_array[idx] = gs[1]
        g2_array[idx] = gs[2]
    end
    fig, axs = subplots(1,2,figsize=(15,5))
    axs[1].plot(alpha_array, g1_array, label="1")
    axs[2].plot(alpha_array, g2_array, label="2")
    symbol = symbols[index]
    axs[1].set_xlabel("$symbol")
    axs[1].set_ylabel("Coupling Strength")
    axs[1].legend()
    axs[2].set_xlabel("$symbol")
    axs[2].set_ylabel("Coupling Strength")
    axs[2].legend()
    savefig("images\\coupling_strengths\\$symbol.png")
end

coupling_strengths (generic function with 1 method)

In [8]:
# symbols = [ωc  g1  g2  λ1  λ2  λc  α1  α2  β]

# for i in 4:5
#     coupling_strengths(i)
# end